In [12]:
import cv2
import mediapipe as mp
import time
import numpy as np
import keras
import matplotlib.pyplot as plt
import pyautogui as p

frameWidth = 1280
frameHeight = 720
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
mpHands=mp.solutions.hands
hands=mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
handsModule = mp.solutions.hands

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(30,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


pTime = 0
cTime = 0
model = keras.models.load_model(r"finalcnn.h5")
with handsModule.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=1) as hands:
    while True:
    #num_imgs_taken=0
    #while num_imgs_taken < 180:
        success, img = cap.read()
        img= cv2.flip(img,1)
        img_w = np.empty(img.shape)
        img_w.fill(255)
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(imgRGB)
        results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        l=[]
        if results.multi_hand_landmarks!= None:
            for handLms in results.multi_hand_landmarks: 
            #handLMs are 21 points. so we need conection too-->mpHands.HAND_CONNECTIONS
                for id, lm in enumerate(handLms.landmark):
                    #print(id, lm)
                    #lm = x,y cordinate of each landmark in float numbers. lm.x, lm.y methods
                    #So, need to covert in integer
                    h, w, c =img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    #print(id, cx, cy)
                    # if id == 4: #(To draw 4th point)
                    cv2.circle(img_w, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
                    mpDraw.draw_landmarks(img_w, handLms, mpHands.HAND_CONNECTIONS) #drawing points and lines(=handconections)
            for point in handsModule.HandLandmark:
 
                  normalizedLandmark = handLms.landmark[point]
                  pixelCoordinatesLandmark = mpDraw._normalized_to_pixel_coordinates(normalizedLandmark.x, normalizedLandmark.y,p.size()[0],p.size()[1])
                  l.append(pixelCoordinatesLandmark)
            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime
            cv2.putText(img_w, "FPS= " + str(int(fps)), (10, 20), cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 1)
           #cv2.imshow('image', img_w)
            img=img_w
            #print(img_w.shape)
           # plotImages(img_w)
            img = cv2.resize(img, (64,64))
            img = np.expand_dims(img, axis=0) 
            word_dict = {0:'left',1:'palm',2:'right',3:'scroll_down',4:'scroll_up',5:'screenshot'}
            predictions = model.predict(img, verbose=0)

            for ind, i in enumerate(predictions):
                   k=(word_dict[np.argmax(i)])

            cv2.putText(img_w, k, (20, 40), cv2.FONT_HERSHEY_PLAIN, 1,(0, 0, 255), 2)
            cv2.imshow('image', img_w)
            if(type(l)!=None):
                p.moveTo(l[9][0],l[9][1])
            if(k=='left'):
                p.click(button='left')
                time.sleep(0.5)
            elif(k=='right'):
                p.click(button='right')
                time.sleep(0.5)
            elif(k=='scroll_up'):
                p.scroll(15)
            elif(k=='scroll_down'):
                p.scroll(-15)
            elif(k=='screenshot'):
                p.screenshot('foo.png')
      #      else:
      #          p.mouseUp(button='left', x=l[9][0], y=l[9][1])
        if cv2.waitKey(1)==27:
            break
    #bgfbhnhyhyhy
cv2.destroyAllWindows()
cap.release()

In [10]:
cv2.destroyAllWindows()
cap.release()